# Set the environment

In [1]:
# set the envoiroment
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import glob
import escher
from escher import Builder
from utils import show_map
from utils.check_precursor_problem import check_precursor_problem

# Read the model published by Schomburg group in 2016
- The model was download from the [website](https://fairdomhub.org/models/225).

In [2]:
model = cobra.io.read_sbml_model('../models/Sulfolobus-solfataricus_D-glucose.xml')# Import the model.
model.solver = "glpk"
model.reactions.get_by_id("T_flux_beta__D__glucose").bounds = (-1,1000) # Set the bounds to the glucose exchange reaction

# The following code is used to change the reactions that are not right.

In [3]:
# The Stoichiometry of the proton of the following two reactions are not right, as I cannot change it directly so I first remove from the model and then add the reaction to the model again with the change of the Stoichiometry.
# First remove the reations that are not right. 
model.reactions.get_by_id('carb_degra_1.2.1.26_R01752').remove_from_model()
model.reactions.get_by_id('oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240').remove_from_model()
model.reactions.get_by_id('oxp_redox_1.10.3.13_soxM_TRANS__RXN__240').remove_from_model()
model.reactions.get_by_id('oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240').remove_from_model()
model.reactions.get_by_id('oxp_3.6.3.14_ATP__synthase').remove_from_model()
# Add the metabolite psi in the model.
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
# Add the reactions again. 
reaction = Reaction('carb_degra_1.2.1.26_R01752')
reaction.name = 'carb_degra_1.2.1.26_R01752'
reaction.lower_bound = -1000. # This is the default
reaction.upper_bound = 1000. # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({"D__glyceraldehyde": -1.0,
                         "H2O": -1.0,
                         "NADP+": -1.0,
                         "H+": 2.0,
                         "D__glycerate": 1.0,
                         "NADPH": 1.0})

reaction = Reaction('oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240')
reaction.name = 'oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({"oxygen": -0.5,
                         "representative_quinol": -1.0,
                         "H+": -4.0,
                         Psi: 4.0,
                         "H2O": 1.0,
                         "representative_quinone": 1.0,
                         "H+_ex": 4.0})

reaction = Reaction('oxp_redox_1.10.3.13_soxM_TRANS__RXN__240')
reaction.name = 'oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({"oxygen": -0.5,
                         "representative_quinol": -1.0,
                         "H+": -2.0,
                         Psi: 2.0,
                         "H2O": 1.0,
                         "representative_quinone": 1.0,
                         "H+_ex": 2.0})

reaction = Reaction('oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240')
reaction.name = 'oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({"oxygen": -0.5,
                         "representative_quinol": -1.0,
                         "H+": -2.0,
                         Psi: 2.0,
                         "H2O": 1.0,
                         "representative_quinone": 1.0,
                         "H+_ex": 2.0})

reaction = Reaction('oxp_3.6.3.14_ATP__synthase')
reaction.name = 'oxp_3.6.3.14_ATP__synthase'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({Psi: -4.0,
                         "H+_ex": -4.0,
                         "ADP": -1.0,
                         "phosphate": -1.0,
                          "ATP": 1.0,
                         "H2O": 1.0,
                         "H+": 3.0})


model.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-2.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 2.0,
                          "H+_ex":2.0})
# reaction = Reaction('aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN')
# reaction.name = 'aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN'
# reaction.lower_bound = 0.  # This is the default
# reaction.upper_bound = 1000.  # This is the default
# model.add_reactions([reaction])
# reaction.add_metabolites({"L__serine":-1.0,
#                          "pyruvate":1.0,
#                          "ammonia": 1.0})

model.reactions.get_by_id("carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NADP").bounds = (-1000,1000)
model.reactions.get_by_id("carb_glucneo_1.2.1.13_GAPDHSYNEC__RXN_NAD").bounds = (-1000,1000)


In [4]:
# Overview of sulfolobus solfataricus map
df_overview_of_map = pd.DataFrame(columns=["Reactions","Metabolites","Compartment"])
df_overview_of_map.loc["overview"] = [len(model.reactions),len(model.metabolites),len(model.compartments)]
df_overview_of_map

,Reactions,Metabolites,Compartment
overview,992,866,2


In [5]:
cobra.io.save_json_model(model, "../models/updated_model.json")

In [6]:
df_overview_of_the_medium = pd.DataFrame(columns=["Reactions","lower_bound"])
df_overview_of_the_medium["Reactions"] = [rxn for rxn in model.medium]
df_overview_of_the_medium["lower_bound"] = [-model.medium[rxn] for rxn in model.medium]
df_overview_of_the_medium

,Reactions,lower_bound
0,T_flux_Ca2+,-1000
1,T_flux_Na+,-1000
2,T_flux_dextrin,-1000
3,T_flux_Co2+,-1000
4,T_flux_Cu2+,-1000
5,T_flux_Fe3+,-1000
6,T_flux_nitrate,-1000
7,T_flux_CO2,-1000
8,T_flux_H+,-1000
9,T_flux_H2O,-1000
